In [1]:
import os
import re
import pickle

import numpy as np
import pandas as pd

In [2]:
import torch
print(torch.__version__)

2.5.1+cu118


In [3]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)
print("CUDA devices:", [torch.cuda.device(i) for i in range(torch.cuda.device_count())])
print("CUDA_VISIBLE_DEVICES:", os.environ.get('CUDA_VISIBLE_DEVICES'))

# Try to allocate a tensor on GPU
try:
    device = torch.device('cuda:0')
    torch.tensor([1.0], device=device)
    print("Successfully allocated tensor on GPU.")
except Exception as e:
    print("Failed to allocate tensor on GPU:", e)

Is CUDA available: True
CUDA device count: 1
CUDA version: 11.8
CUDA devices: [<torch.cuda.device object at 0x14af18d18df0>]
CUDA_VISIBLE_DEVICES: MIG-aa265898-f38d-56fb-a517-3ec989e16b08
Successfully allocated tensor on GPU.


In [4]:
import sys
sys.path.insert(0, '/n/home09/pren/3DGeno_tools/src/gt3d/modules')
# sys.path.append('/n/home09/pren/Higashi')
# sys.path = ['/n/home09/pren/3DGeno_tools/src/gt3d/modules'] + sys.path
sys.path


['/n/home09/pren/3DGeno_tools/src/gt3d/modules',
 '/n/home09/pren/.conda/envs/gt3d/lib/python310.zip',
 '/n/home09/pren/.conda/envs/gt3d/lib/python3.10',
 '/n/home09/pren/.conda/envs/gt3d/lib/python3.10/lib-dynload',
 '',
 '/n/home09/pren/.conda/envs/gt3d/lib/python3.10/site-packages',
 '/n/home09/pren/3DGeno_tools/src']

In [5]:
import higashi
print(higashi.__file__)

/n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/__init__.py


In [6]:
from higashi.Higashi_wrapper import *

In [7]:
fish_path = '/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/multiplexed_fish'

In [8]:
cell_df = pd.read_csv(os.path.join(fish_path, '4DNESMTNNB3N/4DNFIA7FUW8Y.csv'), skiprows=22)
cell_df = cell_df.rename(columns={'##columns=(Cell_ID': 'Cell_ID', 'Zfp804b)': 'Zfp804b'})
cell_df


,Cell_ID,FOV_ID,cell_volume_from_merlin,cell_center_x_global,cell_center_y_global,RNA_experiment_ID,Sample_ID,cluster_subclass,cluster_class,neuron_identity,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,103489270985795086009915676188840872439,0,403.121799,-5984.643199,3962.926100,20220304,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,113811790893231448949429397438012891173,0,1345.443053,-6046.845801,3943.378100,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,119547840309256440899855144626134392385,0,866.273544,-6009.207800,3915.406099,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,120849314423504941338008541020153695260,0,1186.181156,-6114.885802,3938.572100,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,125550627244524118102850004242680097855,0,281.811702,-6190.485804,3985.876101,20220304,C57BL/6_M_1_MOp_1,Peri,Peri,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,89740340849279220555487673174291177981,167,746.108561,3028.620050,-2372.662093,20220304,C57BL/6_M_1_MOp_1,L5/6 NP,Gluta,Neuronal,...,5.0,0.0,0.0,1.0,1.0,0.0,0.0,8.0,0.0,1.0
17852,90550019712359509388050476283813007625,167,931.565237,3093.798051,-2484.658096,20220304,C57BL/6_M_1_MOp_1,L6 CT,Gluta,Neuronal,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,30.0,0.0,0.0
17853,91790428117198932838975210302149417238,167,146.709916,3073.926051,-2429.902094,20220304,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17854,95847719459140129344505526245510134602,167,264.662706,3019.332049,-2441.458095,20220304,C57BL/6_M_1_MOp_1,L6 CT,Gluta,Neuronal,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,1.0


In [9]:
cell_ids = [re.search('cell\d+', filename).group()[4:] for filename in os.listdir(os.path.join(fish_path, 'merfish_contacts_thresh500nm'))]
len(cell_ids)

2420

In [10]:
training_data_meta = pd.DataFrame(cell_ids, columns=['Cell_ID']).merge(cell_df, on='Cell_ID', how='left')
training_data_meta

,Cell_ID,FOV_ID,cell_volume_from_merlin,cell_center_x_global,cell_center_y_global,RNA_experiment_ID,Sample_ID,cluster_subclass,cluster_class,neuron_identity,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,100143354915909514340563966591344282617,84.0,852.211658,2475.098099,-1024.199995,20220304.0,C57BL/6_M_1_MOp_1,L2/3 IT,Gluta,Neuronal,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,33.0,0.0,1.0
1,100444929727335622562402977796883679092,8.0,844.089198,-5802.457800,4135.278100,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100636001562801092372127493520169017848,75.0,598.349001,-6372.189898,5324.277907,20220304.0,C57BL/6_M_1_MOp_1,VLMC,VLMC,Non-Neuronal,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100641667178165970074422975257012665391,84.0,762.882093,2494.970100,-1057.625996,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,2.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,100645494773040598441840741335181821324,81.0,917.879399,2390.604102,-437.051995,20220304.0,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,99732441040257909568179107546777692281,85.0,1189.559867,2588.552102,-950.289998,20220304.0,C57BL/6_M_1_MOp_1,Sst,GABA,Neuronal,...,13.0,2.0,6.0,7.0,1.0,1.0,13.0,17.0,1.0,0.0
2416,99760521419024717766427788907150197811,84.0,554.913079,2633.588103,-1030.841995,20220304.0,C57BL/6_M_1_MOp_1,L5 IT,Gluta,Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2417,99806661526131685558261006620437651282,72.0,625.749255,-6655.781900,5085.343906,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2418,99895158179073067511837782015760741030,9.0,498.213556,-5864.665801,3896.452099,20220304.0,C57BL/6_M_1_MOp_1,Micro,Micro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
label_info = {k:np.asarray(training_data_meta[k]) for k in training_data_meta.columns}
label_info_path = '/n/home09/pren/higfiles/merfish_metadata/label_info_500nmthresh.pickle'
with open(label_info_path, 'wb') as handle:
    pickle.dump(label_info, handle)

In [12]:
config = "/n/home09/pren/higfiles/configs/config_liu_zhuang_t5_500nmthresh.JSON"
config_info = {
    "data_dir": fish_path, # where the data is, the data has to be named as data.txt (v1) or filelist.txt (v2) or the code can't find it
    "label_path": label_info_path,
    "structured": True,
    "input_format": 'higashi_v2',
    "temp_dir": "/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embeddings_liu_zhuang_temp5_500nmthresh/", # where to store model temp files
    "genome_reference_path": "/n/home09/pren/higfiles/params/chromInfo.txt",
    "cytoband_path": "/n/home09/pren/higfiles/params/cytoBand.txt",
    "chrom_list": ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr17', 'chr18', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9',
       'chr16', 'chr19', 'chr2', 'chr8', "chrX"],
    "resolution": 2500000, # 2.5 Mbp
    "resolution_cell": 2500000,
    "resolution_fh": [2500000],
    "embedding_name": "test_dnamerfish_500nmthresh",
    "minimum_distance": 2500000,
    "maximum_distance": -1,
    "local_transfer_range": 0,
    "loss_mode": "zinb",
    "dimensions": 100, # can be adjusted later
    "impute_list":['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr17', 'chr18', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9',
       'chr16', 'chr19', 'chr2', 'chr8', "chrX"],
    "neighbor_num": 5,
    "cpu_num": 10,
    "gpu_num": 1,
    "embedding_epoch":65, # this can be adjusted
    "correct_be_impute": True,
    "header_included": True,
    # "reprocess": True,
    # "contact_header": ['cell_id', 'chrom1', 'pos1', 'chrom2', 'pos2', 'count']
}

# save the config file to the current directory
import json
with open(config, "w") as f:
    json.dump(config_info, f, indent = 6)

In [13]:
# Initialize the Higashi instance
higashi_model = Higashi(config)


In [14]:
# Data processing (only needs to be run for once)
higashi_model.process_data()



[12/07/24 14:29:13] INFO     generate_chrom_start_end Found cached chromosome start/end dict:         ]8;id=878833;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=725738;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#90\90]8;;\
                             /n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embeddings_liu_zhu              
                             ang_temp5_500nmthresh/chrom_start_end.npy                                             

                    INFO     extract_table Found cached data tables:                                 ]8;id=798347;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=131498;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#225\225]8;;\
                                     /n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embedding               
                             s_liu_zhuang_temp5_500nmthresh/data.npy                                               
                                     /n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embedding               
                             s_liu_zhuang_temp5_500nmthresh/weight.npy                                             

                    INFO     create_matrix Generating contact maps for baseline                      ]8;id=839341;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=902876;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#511\511]8;;\

Output()

                    INFO     create_matrix Data loaded                                               ]8;id=27156;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=390679;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#516\516]8;;\

                    INFO     create_matrix Data size: 796,458                                        ]8;id=980011;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=466925;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#535\535]8;;\

                    INFO     create_matrix Found cached data with full set of keys (<HDF5 file       ]8;id=616051;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=690458;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#560\560]8;;\
                             "node_feats.hdf5" (mode r+)>):                                                        
                             /n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embeddings_liu_zh               
                             uang_temp5_500nmthresh/node_feats.hdf5                                                

In [15]:
higashi_model.prep_model()

                    INFO     fetch_info_from_config #CPU: 10                                 ]8;id=263816;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=630123;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#607\607]8;;\

                    INFO     get_free_gpu setting to gpu:0                                    ]8;id=80544;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=304556;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#68\68]8;;\

                    INFO     fetch_info_from_config Training on data from: ['chr1', 'chr10', ]8;id=237411;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=349267;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#627\627]8;;\
                             'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr17', 'chr18',                        
                             'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9', 'chr16',                              
                             'chr19', 'chr2', 'chr8', 'chrX']                                                      

                    INFO     prep_model total_sparsity_cell = 4.2570400800323534e-05         ]8;id=762855;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=544661;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#811\811]8;;\

                    INFO     prep_model No contractive loss                                  ]8;id=132275;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=823704;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#823\823]8;;\

...

                    INFO     generate_attributes Start making attribute                      ]8;id=711194;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=874187;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#756\756]8;;\

Output()

[12/07/24 14:29:24] INFO     fit loss: 0.8282062411308289 loss_best: 0.7962300777435303 epochs: 123  ]8;id=660192;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_backend/Modules.py\Modules.py]8;;\:]8;id=316511;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_backend/Modules.py#334\334]8;;\

                    INFO     __init__ Initializing data generator                                   ]8;id=307814;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_backend/Modules.py\Modules.py]8;;\:]8;id=107919;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_backend/Modules.py#1177\1177]8;;\

                    INFO     __init__ Initializing data generator                                   ]8;id=257237;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_backend/Modules.py\Modules.py]8;;\:]8;id=982334;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_backend/Modules.py#1177\1177]8;;\

In [ ]:
# Stage 1 training
higashi_model.train_for_embeddings()

                    INFO     train_for_embeddings First stage training                      ]8;id=409926;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=360457;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#1596\1596]8;;\

                    INFO     train [ Epoch 0 of 65 ]                                        ]8;id=959425;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=314565;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#1245\1245]8;;\

Output()

[12/07/24 14:31:13] INFO     train  - (Train) bce:  0.4374, mse:  0.8569,  acc: 83.771 %,   ]8;id=122345;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=359545;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#1258\1258]8;;\
                             pearson: 0.439, spearman: 0.431, elapse: 108.946 s                                    

Output()

...

Output()

                    INFO     train  - (Valid) bce:  0.3723,  acc: 88.001 %,pearson: 0.606,  ]8;id=874886;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=477431;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#1277\1277]8;;\
                             spearman: 0.547,elapse: 0.052 s                                                       

                    INFO     train update_rate: 0.003154     0.125972                       ]8;id=585182;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=113135;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#1323\1323]8;;\

                    INFO     train [ Epoch 42 of 65 ]                                       ]8;id=237760;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py\Higashi_wrapper.py]8;;\:]8;id=539295;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Higashi_wrapper.py#1245\1245]8;;\

Output()

In [ ]:
training_data_meta['cluster_subclass'].unique()

In [ ]:
training_data_meta['cluster_class'].unique()

In [ ]:
# Visualize initial embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from umap import UMAP
import seaborn as sns
import matplotlib.pyplot as plt

vec = UMAP(n_components=2, n_neighbors=5, random_state=0).fit_transform(cell_embeddings)
# cell_type = higashi_model.label_info['cluster_subclass']
cell_type = higashi_model.label_info['cluster_class']
batch = higashi_model.label_info['Sample_ID']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
print('vec: ', vec)
print('type(vec): ', type(vec))
print('vec.shape: ', vec.shape)
# sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=5, alpha=0.8, linewidth=0,
#                 hue_order=['L2/3 IT', 'Endo', 'VLMC', 'Astro', 'Oligo', 'Sst', 'OPC', 'L5 IT',
#                            'L4/5 IT', 'Vip', 'L5 ET', 'Micro', 'Pvalb', 'L6 IT', 'L6 CT',
#                            'SMC', 'Peri', 'Lamp5', 'L6b', 'L5/6 NP', 'other', 'Sncg'])
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=5, alpha=0.8, linewidth=0,
                hue_order=['Gluta', 'Endo', 'VLMC', 'Astro', 'Oligo', 'GABA', 'Micro', 'SMC',
                           'Peri', 'other'])
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=batch, ax=ax, s=5, alpha=0.8, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.show()


In [ ]:
# np.save('/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/test2/state/embeddings_dnamerfish_stage1_r2.npy', cell_embeddings)

In [ ]:
higashi_model.current_device

In [ ]:
# higashi_model.higashi_model = torch.load(higashi_model.save_path + "_stage1_model", map_location=higashi_model.current_device)
# higashi_model.node_embedding_init = None


In [ ]:
higashi_model.train_for_imputation_nbr_0()
higashi_model.impute_no_nbr()

In [ ]:
higashi_model.train_for_imputation_with_nbr()
higashi_model.impute_with_nbr()

In [ ]:
# Visualize final embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from sklearn.decomposition import PCA

cell_type = higashi_model.label_info['cluster_class']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
vec = PCA(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
vec = UMAP(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.savefig('/n/home09/pren/figures/dnamerfish_pca_umapp_500nmthresh.png')
plt.show()

In [ ]:
# Visualize final embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from sklearn.decomposition import PCA

cell_type = higashi_model.label_info['neuron_identity']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
vec = PCA(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
vec = UMAP(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.savefig('/n/home09/pren/figures/dnamerfish_pca_umapp_neuronlabels_500nm_thresh.png')
plt.show()

In [ ]:
# ori.data

In [ ]:
count = 0
fig = plt.figure(figsize=(6, 2*5))
i = 0
for id_ in np.random.randint(0, 620, 620):
    if i == 5:
        break
    ori, nbr0, nbr5 = higashi_model.fetch_map("chr3", id_)
    # print('ori.data.shape: ', ori.data.shape)
    if ori.data.shape[0] == 0:
        continue
    else:
        i += 1
    count += 1
    ax = plt.subplot(5, 3, count * 3 - 2)
    ax.imshow(ori.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(ori.data, 0.6))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("raw")
    ax.set_ylabel(f'Cell {id_}')

    
    ax = plt.subplot(5, 3, count * 3 - 1)
    ax.imshow(nbr0.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr0.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=0")
    
    ax = plt.subplot(5, 3, count * 3)
    ax.imshow(nbr5.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr5.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=5")

plt.tight_layout()
# plt.suptitle('Chromosome 3')

plt.savefig('/n/home09/pren/figures/dnamerfish_imputation_500nmthresh.png')